In [ ]:
import pandas as pd
import numpy as np
#import plotly.express as px

## Load & Parse EMS Data

In [ ]:
ems_src = "https://data.cityofnewyork.us/resource/76xm-jjuj.json?$limit=243979&%24where=cad_incident_id%20%25%2095%20%3D%200"
ems_df = pd.read_json(ems_src)
print(ems_df.columns)
ems_df

In [ ]:
# adding basic filters
target_cols = ["incident_travel_tm_seconds_qy", "incident_datetime", "zipcode"]
df = ems_df[target_cols].dropna()
df = df.rename(columns={"incident_travel_tm_seconds_qy": "travel_time"})

# specifying types
df["zipcode"] = df["zipcode"].astype("category")
df["travel_time"] = pd.to_numeric(df["travel_time"])
df["incident_datetime"] = pd.to_datetime(df["incident_datetime"])

# splitting datetime
df["date"] = pd.to_datetime(df["incident_datetime"]).dt.date
df["time"] = pd.to_datetime(df["incident_datetime"]).dt.time
df["time_block"] = df["time"].apply(lambda x: x.hour)
df = df.drop(columns=["incident_datetime"])
df

## Load and Parse Traffic Data

In [ ]:
traffic_src = "https://data.cityofnewyork.us/resource/btm5-ppia.json?$limit=42800"
traffic_df = pd.read_json(traffic_src)
traffic_df["date"] = pd.to_datetime(traffic_df["date"])
print(traffic_df.columns)
traffic_df

In [ ]:
new_cols = ["time_block", "date", "zipcode", "traffic_volume"]
new_data = dict([[name, []] for name in new_cols])
for i, row in traffic_df.iterrows():
    for j in range(7,len(row)):  # 7 to 30
        new_data["date"].append(row[6])
#         location_data = [name.lower().strip() for name in row[2:5]]
        new_data["zipcode"].append(row[2].lower().strip())
        new_data["traffic_volume"].append(row[j])
        new_data["time_block"].append(j-7)

In [ ]:
tvol_df = pd.DataFrame(new_data)
tvol_df

In [ ]:
# save streets for testing street to zip
# example_streets = pd.DataFrame(list(tvol_df["zipcode"].unique()), columns=["streets"])
# example_streets["zipcodes"] = [""]*len(example_streets)
# street_loc = "data/example_streets.csv"
# example_streets.to_csv(street_loc, index=False)  # np.savetxt(street_loc, example_streets, delimiter=",", fmt='%s')
# print("streets saved for later at:", street_loc)
# example_streets

In [ ]:
zip_code_data = pd.read_csv('/kaggle/input/example-streets/example_streets.csv')

In [ ]:
lookup_dict = dict(zip(zip_code_data['streets'], zip_code_data['zipcodes']))

# Map the values in the 'zipcode' column using the dictionary
tvol_df['zipcode'] = tvol_df['zipcode'].map(lookup_dict)

In [ ]:
zip_code_data.head()

In [ ]:
tvol_df.head()

In [ ]:
tvol_df.isnull().sum()

In [ ]:
tvol_df = tvol_df.dropna()

In [ ]:
tvol_df.isna().sum()

In [ ]:
# fix zipcode to int
tvol_df["zipcode"] = pd.to_numeric(tvol_df["zipcode"].apply(lambda x:int(x)))

# standardize datetime stuff
tvol_df["date"] = pd.to_datetime(tvol_df["date"]).dt.date

tvol_df

In [ ]:
# standardizing zipcode
df["zipcode"] = df["zipcode"].apply(lambda x: int(x.replace(',', '')))
df["zipcode"] = df["zipcode"].astype(int)

# reminder of ems data
df

In [ ]:
tvol_df['zipcode']

In [ ]:
tvol_df.dtypes

## Merging the Dataset

In [ ]:
limited_tvol_df = tvol_df.head(80000)
#limited_df = df.head(10000)


In [ ]:

# Assuming tvol_df is your DataFrame with the columns "time_block", "date", "zipcode", "traffic_volume"
# Replace this line with the actual data loading step if needed

# Sample data loading
# tvol_df = pd.read_csv("your_data.csv")

# Assuming you have a DataFrame named tvol_df
# Replace "your_data.csv" with the actual file path if you are loading data from a CSV file

# Use groupby to get unique combinations of date, time_block, and zipcode
unique_combinations = tvol_df.groupby(["date", "time_block", "zipcode"]).size().reset_index(name='count')

# Sample 100,000 records based on unique combinations
sampled_combinations = unique_combinations.sample(n=50000, random_state=42)

# Filter the original DataFrame based on the sampled combinations
sampled_records = tvol_df[tvol_df.set_index(["date", "time_block", "zipcode"]).index.isin(sampled_combinations.set_index(["date", "time_block", "zipcode"]).index)]

# Display or use the sampled records as needed
print(sampled_records)

In [ ]:
sampled_records.sort_values(by=["date", "time_block", "zipcode"], inplace=True)

In [ ]:
from tqdm import tqdm  # progress bar since this is a big database operation

found_counter = 0
notfound_counter = 0

merge_columns = ["time_block", "date", "zipcode", "traffic_volume", "avg_travel_time"]
merge_data = []
for i, row in tqdm(sampled_records.iterrows(), total=sampled_records.shape[0]):  # TODO: fix date-time comparison
    related_ems = df[
                    (df["date"] == row["date"]) &
                    (df["zipcode"] == row["zipcode"])]
    avg_travel = pd.NA
    if len(related_ems) > 0:
        avg_travel = related_ems["travel_time"].mean()
        found_counter += 1
        print("Travel time found at:", row)  # this means its working and we are finding matches
        print("Travel time is:",related_ems["travel_time"])  # this means its working and we are finding matches

    else:
        notfound_counter += 1
    add_data = [row["time_block"], row["date"], row["zipcode"], row["traffic_volume"], avg_travel]
    merge_data.append(add_data)
merge_data  # 4% per hour?

In [ ]:
# validate the percentage of the database that was successfully merged
found_perc = found_counter / (found_counter + notfound_counter)
print("Found ratio:", found_counter, "of", (found_counter + notfound_counter), "records")
print(found_perc*100, "%")

In [ ]:
merged_df = pd.DataFrame(merge_data, columns=merge_columns)
merged_df

In [ ]:
# confirm that some of the entries were written
merged_df[~merged_df["avg_travel_time"].isna()]

In [ ]:
merge_loc = "final_merged_data.csv"
merged_df.to_csv(merge_loc, index=False)
print("streets saved for later at:", merge_loc)

In [ ]:
result_df = merged_df[~merged_df["avg_travel_time"].isna()].groupby(['date', 'zipcode', 'time_block']).agg({
    'traffic_volume': 'mean',  # You can use 'mean', 'median', or any other desired aggregation function
    'avg_travel_time': 'mean'  # Similarly, choose the aggregation function for this column
}).reset_index()

In [ ]:
print(result_df)


In [ ]:
res = "final_merged_data_grouped.csv"
result_df.to_csv(res, index=False)
#print("streets saved for later at:", merge_loc)

## Analysis of Data Merged

In [ ]:
from datetime import datetime

df = pd.read_csv("data/new_merged_data.csv")
# processing again
df["date"] = df["date"].apply(lambda this_date: datetime.fromisoformat(this_date))
df

In [ ]:
df.describe()

In [ ]:
fig = px.line(df, x="date", y="avg_travel_time")
fig.show()

In [ ]:
datena = {}
for i, row in df.iterrows():
    year = row["date"].year
    if year not in datena.keys():
        datena[year] = [0,0]
    datena[year][0] += 1  # raw count
    if pd.isna(row["avg_travel_time"]):
        datena[year][1] += 1
        
translation = []
for year, count in datena.items():
    translation.append([year, count[0], "Total"])
    translation.append([year, count[1], "Null"])
    translation.append([year, count[0]-count[1], "NotNull"])
datena_df = pd.DataFrame(translation, columns=["Year", "Count", "Type"])
datena_df.head()

In [ ]:
fig = px.bar(datena_df, x="Year", y="Count", color="Type", barmode='group')
fig.show()

In [ ]:
fig = px.bar(datena_df[datena_df["Type"]=="NotNull"], x="Year", y="Count")
fig.show()

In [ ]:
def get_null_dist(this_df, target_var="time_block", name="Time", addTotal=False):
    this_df.value_counts().to_dict()
#     nadata = {}  # 
#     for i, row in this_df.iterrows():
#         var = row[target_var]
#         if var not in nadata.keys():
#             nadata[var] = [0,0]
#         nadata[var][0] += 1  # raw count
#         if pd.isna(row["avg_travel_time"]):
#             datena[var][1] += 1

#     print(nadata)
#     translation = []
#     for key, val in nadata.items():
#         if addTotal:
#             translation.append([var, count[0], "Total"])
#         translation.append([var, count[1], "Null"])
#         translation.append([var, count[0]-count[1], "NotNull"])
#     na_df = pd.DataFrame(translation, columns=[name, "Count", "Type"])
    return na_df

timena_df = get_null_dist(df, "time_block", "Time")
timena_df.head()

In [ ]:
fig = px.bar(timena_df, x="Time", y="Count", color="Type", barmode='group')
fig.show()

In [ ]:
df["zipcode"].value_counts()